In [76]:
import numpy as np
import pandas as pd 
pd.set_option("max_columns",None)
from sklearn.linear_model import LinearRegression, Ridge

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import Lasso, Ridge, SGDRegressor,LinearRegression,RidgeCV,ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,RobustScaler
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score

pd.set_option('max_columns', 200)



In [71]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

In [77]:
#data initilization
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]

weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
ali = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
dataset = ali.drop(["name","date","lat","lon"],axis=1)

In [78]:
#encoding for not numerical datas 
dataset= dataset.set_index(["Date"])
year = pd.DataFrame(data=pd.get_dummies(dataset.index.year,prefix="year"))
month = pd.DataFrame(data=pd.get_dummies(dataset.index.month, prefix="month"))
#day = pd.DataFrame(data=pd.get_dummies(dataset.index.day,prefix="day"))
hour = pd.DataFrame(data=pd.get_dummies(dataset.index.hour,prefix="hour"))
#frames = [year,month,day,hour]
frames = [year,month,hour]
onehot_encoded = ali.join(frames)



In [79]:
#dropping useless features
onehot_encoded = onehot_encoded.drop(["Date","name","date","lat","lon"],axis=1)

In [80]:
#normalization in range (-1,1)
def normalize(df,column_list):
    for col in column_list:
        feature_range = (-1,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [85]:
normalized = normalize(onehot_encoded,onehot_encoded.columns[0:11])


In [145]:
#choosing specific year for train data, so other years can be drop
target = normalized.loc[normalized.year_2021 == 1]
target = target.drop(["year_2020","year_2022","year_2021"],axis=1)
target = target.reset_index(drop=True)

In [153]:
target

,Production,temperature,cloud cover,global_rad:W,diffuse_rad:W,direct_rad:W,relative_humidity_2m:p,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,sun_elevation:d,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,-1.0,-0.332143,0.240,-1.0,-1.0,-1.0,0.482650,-0.571429,-1.0,-0.384155,-0.985333,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.0,-0.317857,-0.050,-1.0,-1.0,-1.0,0.421661,-0.653061,-1.0,-0.360239,-0.872000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.0,-0.317857,0.008,-1.0,-1.0,-1.0,0.396425,-0.612245,-1.0,-0.369208,-0.741333,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.0,-0.357143,0.542,-1.0,-1.0,-1.0,0.581493,-0.510204,-1.0,-0.420030,-0.584000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1.0,-0.342857,0.536,-1.0,-1.0,-1.0,0.543638,-0.571429,-1.0,-0.399103,-0.425333,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8416,-1.0,-0.360714,0.000,-1.0,-1.0,-1.0,0.444795,-0.653061,-1.0,-0.402093,-0.252000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8417,-1.0,-0.346429,0.036,-1.0,-1.0,-1.0,0.375394,-0.795918,-1.0,-0.348281,-0.409333,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8418,-1.0,-0.350000,0.066,-1.0,-1.0,-1.0,0.379600,-0.816327,-1.0,-0.351271,-0.568000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8419,-1.0,-0.385714,0.388,-1.0,-1.0,-1.0,0.524711,-0.632653,-1.0,-0.426009,-0.725333,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
"""train_one_year = normalized[8712:17133].reset_index(drop=True)
test_one_week = normalized[17133:17863].reset_index(drop=True)"""

In [146]:
#taking january 2022 as a dataset
#aw = normalized.loc[normalized.year_2022 == 1]

months = []

for i in target.columns[11:23]:
    month = target.loc[target[f"{i}"]== 1].copy()
    aw1 = month.reset_index(drop=True)
    months.append(aw1)

In [160]:
def make_data(dataframe):
    train = dataframe[:len(dataframe)- 24*5]
    test = dataframe[len(dataframe)- 24*5:]


    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('Production')
    y_test = x_test.pop('Production')

    return x_train, x_test, y_train, y_test 

In [187]:


def metrics(y_train1,y_train_pred1,y_test1,y_test_pred1):
    print("for train data, mean squared error is",mean_squared_error(y_train1, y_train_pred1))
    print("for train data, mean absolute error is:",mean_absolute_error(y_train1, y_train_pred1))
    print("for train data, mean absolute percentage error is",mean_absolute_percentage_error(y_train1, y_train_pred1),"\n")

    print("for test data, mean squared error is",mean_squared_error(y_test1, y_test_pred1))
    print("for test data, mean absolute error is:",mean_absolute_error(y_test1, y_test_pred1))
    print("for test data, mean absolute percentage error is",mean_absolute_percentage_error(y_test1, y_test_pred1),"\n")


In [188]:
def calculate_ridge(train_x,test_x,train_y,test_y):
    ridge_reg = Ridge(alpha=1) 

    ridge_reg.fit(train_x,train_y)

    ridge_train_1 = ridge_reg.predict(train_x)
    ridge_test_1 = ridge_reg.predict(test_x)
    print("Ridge Model results")
    metrics(train_y,ridge_train_1,test_y,ridge_test_1)


def calculate_rosso(train_x_2,test_x_2,train_y_2,test_y_2):

    lasso_reg = Lasso(alpha=0.1)
    lasso_reg.fit(train_x_2,train_y_2)
    lasso_train_2 = lasso_reg.predict(train_x_2)
    lasso_test_2 = lasso_reg.predict(test_x_2)
    print("Rosso Model results")
    metrics(train_y_2,lasso_train_2,test_y_2,lasso_test_2)
    

In [190]:
#main loop

for number, month in enumerate(months,start=1):
    print(f"test results for month{number} is ;")

    x_train, x_test, y_train, y_test = make_data(month)

    calculate_ridge(x_train, x_test, y_train, y_test)

    calculate_rosso(x_train, x_test, y_train, y_test)

    print(f"##########################end of test {number}##################################")




test results for month1 is ;
Ridge Model results
for train data, mean squared error is 0.02561728422831954
for train data, mean absolute error is: 0.09375391524498237
for train data, mean absolute percentage error is 0.4144290700771424 

for test data, mean squared error is 0.026917108211472862
for test data, mean absolute error is: 0.09840425124889883
for test data, mean absolute percentage error is 0.3309507519658906 

Rosso Model results
for train data, mean squared error is 0.13211632420662772
for train data, mean absolute error is: 0.27107478836036997
for train data, mean absolute percentage error is 2.0377515752568716 

for test data, mean squared error is 0.09428214622782091
for test data, mean absolute error is: 0.23444744699747153
for test data, mean absolute percentage error is 0.9535429511127725 

##########################end of test 1##################################
test results for month2 is ;
Ridge Model results
for train data, mean squared error is 0.03821794127394319